In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.2'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,252 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,351 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 4,939 kB in 1s (3,35

In [ ]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrame Basics").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://raw.githubusercontent.com/Ekenc/Project4/main/Data/Merged_Target_SeaLevel.csv"
spark.sparkContext.addFile(url)
df = pd.read_csv(url)

# Show DataFrame
df.head()


,Unnamed: 0,Year,Mean Nitrous Oxide Concentration,Mean Carbon Dioxide Concentration,Mean Methane Concentration,Precipitation Anomaly,Mean Adjusted Sea Level (inches),Sea Temperature Anomaly,Earth Surface Temperature Anomaly (land and ocean)
0,0,1964,291.2,319.620000,1260.30,-0.041776,4.169291,-0.546,-0.252
1,1,1967,291.5,322.180000,1284.03,-0.096894,4.452756,-0.420,0.000
2,2,1970,293.8,325.620315,1351.70,-0.070516,4.677165,-0.294,0.108
3,3,1971,294.0,326.320000,1357.20,0.032240,4.881890,-0.510,-0.126
4,4,1972,295.6,328.742110,1380.10,-0.772485,5.240157,-0.186,0.072


In [ ]:
# Define features set
X = df.drop("Mean Adjusted Sea Level (inches)", axis = 1)
X = X.drop("Unnamed: 0", axis = 1)
X.head()

,Year,Mean Nitrous Oxide Concentration,Mean Carbon Dioxide Concentration,Mean Methane Concentration,Precipitation Anomaly,Sea Temperature Anomaly,Earth Surface Temperature Anomaly (land and ocean)
0,1964,291.2,319.620000,1260.30,-0.041776,-0.546,-0.252
1,1967,291.5,322.180000,1284.03,-0.096894,-0.420,0.000
2,1970,293.8,325.620315,1351.70,-0.070516,-0.294,0.108
3,1971,294.0,326.320000,1357.20,0.032240,-0.510,-0.126
4,1972,295.6,328.742110,1380.10,-0.772485,-0.186,0.072


In [ ]:
# Define target vector
y = df["Mean Adjusted Sea Level (inches)"]
y[:48]

0     4.169291
1     4.452756
2     4.677165
3     4.881890
4     5.240157
5     5.003937
6     5.472441
7     5.409449
8     5.303150
9     5.598425
10    6.153543
11    5.748031
12    5.771654
13    5.795276
14    5.980315
15    6.157480
16    6.248031
17    6.346457
18    6.374016
19    6.303150
20    6.507874
21    6.622047
22    6.783465
23    7.059055
24    6.669291
25    7.003937
26    7.055118
27    7.271654
28    7.366142
29    7.728346
30    7.712598
31    7.716535
32    7.885827
33    7.960630
34    8.303150
35    8.531496
36    8.834646
37    8.897638
38    9.244094
39    8.913386
40    8.579036
41    8.924984
42    9.048399
43    9.110986
44    9.234521
45    9.480223
46    9.592477
Name: Mean Adjusted Sea Level (inches), dtype: float64

In [ ]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [ ]:
# Split the dataset using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
                                                                                                

In [ ]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

In [ ]:
# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Create a linear regression model
model = LinearRegression()

In [ ]:
# Train the model
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
# Make predictions
predictions = model.predict(X_test)

In [ ]:
# Calculate the mean squared error, model evaluation
print(
  'mean_squared_error : ', mean_squared_error(y_test, predictions))
print(
  'mean_absolute_error : ', mean_absolute_error(y_test, predictions))


# Compute the linear regression model score
model.score(X_test, y_test)


mean_squared_error :  0.04680229496328442
mean_absolute_error :  0.18341929339919671


0.9785620742423372

In [ ]:
# Display features
model.feature_names_in_

array(['Year', 'Mean Nitrous Oxide Concentration',
       'Mean Carbon Dioxide Concentration', 'Mean Methane Concentration',
       'Precipitation Anomaly', 'Sea Temperature Anomaly',
       'Earth Surface Temperature Anomaly (land and ocean)'], dtype=object)

In [ ]:
# Display the slope
print(f"Model's slope: {model.coef_}")

Model's slope: [ 0.10987448  0.01392843  0.00998927 -0.00279497  0.09777966  1.220876
 -1.33783906]


In [ ]:
# Display the y-intercept
print(f"Model's y-intercept: {model.intercept_}")

Model's y-intercept: -214.77442830922956


In [ ]:
# Display the model's best fit line formula
print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]}X")

Model's formula: y = -214.77442830922956 + 0.10987447748116552X


In [ ]:
# Display the formula
print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]} * 2030")

# Predict the 
y_0 = model.intercept_ + model.coef_[0] *2030

# Display the prediction
print(f": {y_0:.3f}")

Model's formula: y = -214.77442830922956 + 0.10987447748116552 * 2030
: 8.271


AttributeError: ignored